In [10]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import seaborn as sns
from sklearn.model_selection import train_test_split

In [11]:
# Upload file
data = pd.read_csv('../Dataset/emails.csv')
# Select the first two columns
data = data[['Text', 'Spam']]
data.head()

C:\Users\giova\AppData\Local\Temp\ipykernel_20496\1947220157.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Dataset/emails.csv')


,Text,Spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [13]:
print("")
print(f"Data Shape:\n{data.shape}\n")
print("")
print(f"Data Info:\n{data.info()}\n")
print("")
print(f"Count Categorical Values:\n{data.Spam.value_counts()}\n")
print("")
print(f"Any nulls ?:\n{data.isnull().sum()}\n")




Data Shape:
(10847, 2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10847 entries, 0 to 10846
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    10847 non-null  object
 1   Spam    10847 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 169.6+ KB
Data Info:
None


Count Categorical Values:
0    8839
1    2008
Name: Spam, dtype: int64


Any nulls ?:
Text    0
Spam    0
dtype: int64



In [14]:
data.isna().sum()
data.drop_duplicates(inplace=True) # Remove duplicates

In [245]:
data.shape

(10847, 2)

In [15]:
data['Category']=data['Spam'].apply(lambda x:"spam" if x==1 else "ham")
data.tail(100)

,Text,Spam,Category
10747,For the most sparkling shopping breaks from 45...,1,spam
10748,If you hear a loud scream in about &lt;#&gt; m...,0,ham
10749,Ok i thk i got it. Then u wan me 2 come now or...,0,ham
10750,Txt: CALL to No: 86888 & claim your reward of ...,1,spam
10751,U GOIN OUT 2NITE?,0,ham
...,...,...,...
10842,This is the 2nd time we have tried 2 contact u...,1,spam
10843,Will ü b going to esplanade fr home?,0,ham
10844,"Pity, * was in mood for that. So...any other s...",0,ham
10845,The guy did some bitching but I acted like i'd...,0,ham


We can now process our data

In [16]:
string_text=[]

for j in data["Text"]:
    j=j.replace("Subject:", "") # Remove the word Subject: 
    j=j.lower() # Lower case
    j=re.sub("[^a-z0-9 ]","",j) # removing everything except space and a-z
    
    string_text.append(j)
    


In [18]:
data['Text']=string_text # Creating a new column

In [19]:

data.head()

,Text,Spam,Category
0,naturally irresistible your corporate identit...,1,spam
1,the stock trading gunslinger fanny is merril...,1,spam
2,unbelievable new homes made easy im wanting ...,1,spam
3,4 color printing special request additional ...,1,spam
4,do not have money get software cds from here...,1,spam


In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data.Text,data.Spam,test_size=0.25)


In [22]:
#CounterVectorizer Convert the text into matrics
from sklearn.feature_extraction.text import CountVectorizer


Naive Bayes Have three Classifier(Bernouli,Multinominal,Gaussian) Here I use Multinominal Bayes Because here data in a discrete form discrete data(e.g movie ratings ranging 1 to 5 as each rating will have certain frequency to represent)

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

Training Model

In [24]:
clf.fit(X_train,y_train)



Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [25]:
Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

Here I given Two email Two detect 1st One is looking good and the other one looking spam

In [83]:
emails=[
    "Leave ID: 893384029758 Shahie Dervishi's accommodation request is still pending because the colleague has not yet submitted the required documentation. If completed documentation is not submitted on time, the colleague's request may be delayed or denied. You will be notified when a decision has been made. If you have any questions, please contact ReedGroup at (866) 678-7114."
   
]

Predict Email

In [84]:
clf.predict(emails)


array([0], dtype=int64)

Prediction of Model

In [87]:
acc = clf.score(X_test,y_test)
probability_scores = clf.predict_proba(emails)
print(f"acc:{acc}")
# Print the probability scores for each class
for i, probabilities in enumerate(probability_scores):
    print(f"Email {i+1} - Probability Scores:", probabilities)


acc:0.9457964601769911
Email 1 - Probability Scores: [9.99999999e-01 1.42441371e-09]


Save Model

In [88]:
import pickle

def save_model(clf, filename):
    with open(filename, 'wb') as f:
        pickle.dump(clf, f)



In [89]:
save_model(clf,"phishie_1.pkl")



In [97]:
def load_model(model_path):
    with open(model_path, 'rb') as model_file:
        model = pickle.load(model_file)
    return model


def predict_result(model, preprocessed_email):
    # Predict using the model
    predicted_label = model.predict(preprocessed_email)
    probability_scores = model.predict_proba(preprocessed_email)

 
    probability_not_spam = probability_scores[0][0]
    probability_spam = probability_scores[0][1]
    
    is_spam = predicted_label[0] == 1

    
    print(f"probability_not_spam: {probability_not_spam}, probability_spam: {probability_spam}, label: {is_spam}")
    

In [98]:
import pickle

def save_model(clf, filename):
    with open(filename, 'wb') as f:
        pickle.dump(clf, f)



In [101]:
model = load_model('phishie_1.pkl')
predict_result(model,emails)

probability_not_spam: 0.9999999985755608, probability_spam: 1.4244137129791646e-09, label: False
